In [1]:
from scipy import stats
import os
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV, RFE
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV, LogisticRegression, Lasso
import matplotlib.pyplot as plt
import pandas as pd
# from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import sklearn
from sklearn.utils import shuffle

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.svm import SVC
import numpy as np 


In [2]:

def stat_t_u(var_df):
    """
    T检验/U检验: 同时对连续变量进行两个检验
    # T检验分为 1. 单样本T检验（比如抽样100人身高和全国人的平均身高167cm 是否有显著差异）
    #          2. 独立样本T检验 （也叫双样本T检验，比如全班男女身高是否有显著差异，这里的男女是两个独立变量）
    #          3. 联合样本T检验（比如吃饭前后班级同学的血糖浓度有没有变化，是针对同一批患者，且数量必须一致，研究时间等相关性）
    # 检验分为单侧检验和双侧检验，1. 双侧检验是两个变量绝对值的显著性 2.单侧是仅大于或小于的情况
    # 其中独立样本T检验的适用条件为：
    # 独立样本t检验，用于两个完全独立的、符合 正态分布的样本的均数比较。所以使用前先进性正态性和方差齐性检验
    # 根据两样本的方差是否相等，可分为：总体方差相等的t检验总体方差不等的t检验
    # 但是对于大样本，不一定要满足正态性参见《数据不满足正态分布，到底能不能用t检验？》
    
    # Mann-Whitney U 检验是零假设的非参数检验，要求样本 x 的分布与样本 y 的分布相同。
    # U检验不需要数据满足正态分布，而T检验需要满足。

    
    var_df: 输入的DataFrame 包含特征和标签
    return : 返回U检验保留的特征
    """
    # 对连续变量进行 独立样本T检验
    del_Ustat_var, contain_Ustat_var = [], []
    del_Tstat_var, contain_Tstat_var = [], []
    contain_Ustat_p = []
    contain_Tstat_p = []
    
    for column_name in var_df.columns[:-1]:
        # print(column_name, end=' ')
        
        select_df = var_df[[column_name, 'bpCR']].dropna()  # 非空值计算
    #     select_df = var_df[[column_name, 'bpCR']].fillna(var_df[column_name].median)  # 众数填充后计算
    
        Neg_group = select_df[select_df['bpCR'] == 0][column_name].tolist()
        Pos_group = select_df[select_df['bpCR'] == 1][column_name].tolist()
        # print('正态性X1/X2(>0.05表示满足正态性)',stats.shapiro(Neg_group)[1], stats.shapiro(Pos_group)[1])
        equal_var = stats.levene(Neg_group, Pos_group)[1] > 0.05  # 判断是否方差齐性 >0.05是方差齐性的
        
        t_res = stats.ttest_ind(Neg_group, Pos_group, equal_var=equal_var)
        # print('T检验p值',t_res[1])
        u_res = stats.mannwhitneyu(Neg_group, Pos_group)
        # print('U检验p值',u_res[1])
        
        if t_res[1] < 0.05:
            contain_Tstat_var.append(column_name)
            contain_Tstat_p.append(t_res[1])
        else:
            del_Tstat_var.append(column_name)
        if u_res[1] < 0.05:
            contain_Ustat_var.append(column_name)
            contain_Ustat_p.append(u_res[1])
        else:
            del_Ustat_var.append(column_name)    
            
    return contain_Ustat_p, contain_Ustat_var, contain_Tstat_p, contain_Tstat_var
    

In [3]:

def lassocv_selector(contain_stat, X, y):
    """
    LassoCV 特征筛选, 并以LR作为后续分类器评价筛选结果
    """
    alphas = np.logspace(-5, 3, 50)
    lassocv = LassoCV(alphas=alphas,cv=5, max_iter=100000)
    X_std = StandardScaler().fit_transform(X)
    lassocv.fit(X_std, y)
    X_select = X[:, lassocv.coef_ != 0]
    
    # print('删除的特征为：', contain_stat[lassocv.coef_ == 0])
    # print('最佳lambda:', lassocv.alpha_)
    # print('最佳系数:', lassocv.coef_)   

    # 用逻辑回归粗略验证剩余特征的结果
    LR = LogisticRegression(penalty='l2', max_iter=10000)
    print('输入模型的X shape', X_select.shape)
    LR.fit(X_select, y)
    score = cross_val_score(LR, X_select, y, cv=5, scoring='roc_auc').mean()
    print('Score: ', score) 


In [4]:
def ref_selector(estimator, n_features_to_select, contain_stat, X, y):
    """
    ref_selector: 通过REF 特征递归消除法选择最佳的模型
    
    """
    rfe = RFE(estimator=estimator,     # 学习器
                  n_features_to_select=n_features_to_select, 
                  step=1,           # 每次移除特征个数
                  ).fit(X, y)

    X_RFE = rfe.transform(X)
    
    select_features = np.array(contain_stat)[rfe.support_]
    print(select_features)  #  选择的特征名称
    score = cross_val_score(estimator, X_RFE, y, scoring='roc_auc').mean().round(7)
    print(score)
    
    return score, X_RFE, select_features


In [23]:
base_dir = './'

In [24]:

df_adc_zy = pd.read_csv(os.path.join(base_dir, 'zunyi_dec_adc_result.csv'), index_col=0)
df_adc_syf1 = pd.read_csv(os.path.join(base_dir, 'syf_stage1_dec_adc_result.csv'), index_col=0)
df_adc_syf2 = pd.read_csv(os.path.join(base_dir, 'syf_stage2_dec_adc_result.csv'), index_col=0)
df_adc = pd.concat([df_adc_zy, df_adc_syf1, df_adc_syf2])


In [25]:

# sd_dcereg_path = os.path.join(base_dir, 'sd_dec_dcereg_result.csv')
# zy_dcereg_path = os.path.join(base_dir, 'zunyi_dcereg_result.csv')
# syf1_dcereg_path = os.path.join(base_dir, 'syf_stage1_dcereg_result.csv')
# syf2_dcereg_path = os.path.join(base_dir, 'syf_stage2_dcereg_result.csv')

# df_dcereg_sd = pd.read_csv(sd_dcereg_path, index_col=0)
# df_dcereg_zy = pd.read_csv(zy_dcereg_path, index_col=0)
# df_dcereg_syf1 = pd.read_csv(syf1_dcereg_path, index_col=0)
# df_dcereg_syf2 = pd.read_csv(syf2_dcereg_path, index_col=0)
# df_dcereg = pd.concat([df_dcereg_sd, df_dcereg_zy, df_dcereg_syf1, df_dcereg_syf2])


In [27]:
# 读取临床字段表,并将PCR列合并到组学的dataframe中
df_clinical = pd.read_csv('判定_fill_df.csv', index_col=0)
df_clinical.index.name = 'patient_id'
df_pcr = df_clinical[['bpCR']]

In [34]:

df_adc_radiomics = pd.merge(df_adc, df_pcr, on='patient_id')
df_adc_radiomics.shape, df_adc.shape

((687, 1133), (727, 1132))

In [29]:
U_p, U_var, T_p, T_var = stat_t_u(df_adc_radiomics)
merge_var = [i for i in U_var if i in T_var]
U_p, U_var, T_p, T_var,merge_var = np.array(U_p), np.array(U_var), np.array(T_p), np.array(T_var), np.array(merge_var)
select_U_var = U_var[U_p < 0.05].tolist()
select_T_var = T_var[T_p < 0.05].tolist()
len(select_U_var), len(select_T_var)



(220, 166)

In [30]:
# df_adc_radiomics[df_adc_radiomics['bpCR'] == 1].var().tolist()

In [31]:
# select_T_var = df_adc_radiomics.columns.tolist()

In [41]:
# 计算每个特征与目标之间的斯皮尔曼相关系数
correlations = df_adc_radiomics[select_U_var].apply(lambda x: x.corr(df_adc_radiomics['bpCR'], method='spearman'))
# 筛选相关系数大于0.7的特征
selected_features = correlations[correlations > 0.2].index.tolist()
# 打印筛选结果
print("Selected features:", selected_features)

from scipy.stats import spearmanr, pearsonr

for idx in range(len(select_T_var)):
    corr, p_value = spearmanr(df_adc_radiomics[select_T_var[idx]], df_adc_radiomics['bpCR'])
    if abs(corr) > 0.2:
        print(corr)

Selected features: []


In [86]:
def calc_sperman_cor(X, Y):
    return stats.spearmanr(X, Y)[0]
X = [1, 2, 5]
Y = [2, 2, 4]


0.8660254037844387

In [81]:
df_test = df_dce_radiomics[select_T_var + ['bpCR']]
features = df_test.columns[:-1]

# 可以通过调节random_state 的数值，查看输出保留特征的稳定性
shuffle_df = shuffle(df_test, random_state=1)
print('特征总数：', len(shuffle_df.columns[:-1]))

X = shuffle_df.values[:, :-1]
y = shuffle_df.values[:, -1].astype(np.int8)   # 需要转换为int 类型，原来是object类型
lassocv_selector(features, X, y)
LR = LogisticRegression(penalty='l2', max_iter=10000)
print('输入模型的X shape', X.shape)
LR.fit(X, y)
score = cross_val_score(LR, X, y, cv=5, scoring='roc_auc').mean()
print('Score: ', score) 


特征总数： 73


/app/data/anaconda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.033442354850656386, tolerance: 0.0066417241379310345
  model = cd_fast.enet_coordinate_descent_gram(
/app/data/anaconda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.1679540437726601, tolerance: 0.0066417241379310345
  model = cd_fast.enet_coordinate_descent_gram(
/app/data/anaconda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.515938234030294, tolerance: 0.0066417241379310345
  model = cd_fast.enet_coordinate_descent_gram(
/app/data/anaconda/lib/python3.9/site-packages/sklearn/linear_mode

输入模型的X shape (363, 3)
Score:  0.5977641521119782
输入模型的X shape (363, 73)
Score:  0.5599653164870556


In [ ]:
init_C = [0.1, 0.5, 1.0, 1.5]

for C in init_C:
    clf_dict = {
    'SVC': SVC(kernel="linear", C=C, probability=True),  # 0.8
    'LR_2': LogisticRegression(C=C, max_iter=1000), # 0.5
    'LR_1': LogisticRegression(penalty='l1', solver='liblinear', C=C, max_iter=1000),  # 1.6

    # 不使用树模型作为RFE特征选择的基础模型，一方面是由于数模型参数多；另一方面输出的结果不稳定
    # 'GBDT': GradientBoostingClassifier(learning_rate=0.1, n_estimators=100, max_depth=C, random_state=2),  # 50, 2, 2
    # 'RF': RandomForestClassifier(n_estimators=100, max_depth=C, random_state=2),  # 50, 4  2
    # 'XGB': XGBClassifier(learning_rate=0.1, n_estimators=100, max_depth=C, random_state=2),  # 30, 2, 2
    }


    for  clf_name, clf in clf_dict.items():  # 遍历每一个分类器的RFE    

        print('================', clf_name, C, '=============================')
        scores_list = []

        select_features_list = []
        for n_features_to_select in range(1, X.shape[-1] + 1): # 12个可用特征
            score, X_RFE, select_features = ref_selector(clf,  n_features_to_select, features, X, y)
            scores_list.append(score)

            select_features_list.append(select_features)


================ SVC 0.1 =============================
